# 81. Gradio_Stream

## Overview  
This exercise demonstrates how to build a Retrieval-Augmented Generation (RAG) system using Gradio and how to generate and stream responses in real-time using its streaming features. Through this exercise, you will learn to handle real-time interactions with users via a web-based interface. This process helps manage the overall conversation flow, thereby providing more detailed and meaningful responses.
 
## Purpose of the Exercise
The purpose of this exercise is to implement real-time response generation and streaming capabilities using Gradio to develop a live interactive chatbot interface. By the end of this tutorial, users will be able to create a dynamic chat system that streams responses as they are generated, enhancing user engagement and interaction.



In [1]:
!pip install -qU gradio python-dotenv langchain-upstage python-dotenv

In [2]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [12]:
import gradio as gr

from langchain_upstage import UpstageDocumentParseLoader, UpstageGroundednessCheck, ChatUpstage, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.utils.math import cosine_similarity
from langchain.docstore.document import Document


from IPython.display import display, HTML
from tokenizers import Tokenizer

from tokenizers import Tokenizer

llm = ChatUpstage(streaming=True)

In [4]:
# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

In [5]:
doc_loader = UpstageDocumentParseLoader("laws.pdf", output_format='html', coordinates=False)
docs = doc_loader.load()

In [6]:
for doc in docs:
    pprint(doc.page_content[:100])

("<h1 id='0' style='font-size:18px'>경기규칙</h1><br><h1 id='1' "
 "style='font-size:16px'>Laws of the Game</h")


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

print("Splits:", len(splits)) 

Splits: 234


In [18]:
# semantic chunking split
from langchain.docstore.document import Document
def semantic_chunker(docs, min_chunk_size=100, chunk_overlap=10, max_chunk_size=1000, merge_threshold=0.7, embeddings=UpstageEmbeddings(model="solar-embedding-1-large")):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=min_chunk_size, chunk_overlap=chunk_overlap)
    init_splits = text_splitter.split_documents(docs)
    splits = []

    base_split_text = None
    base_split_emb = None
    for split in init_splits:
        if base_split_text is None:
            base_split_text = split.page_content
            base_split_emb = embeddings.embed_documents([base_split_text])[0]
            continue

        split_emb = embeddings.embed_documents([split.page_content])[0]
        distance = cosine_similarity(X=[base_split_emb], Y=[split_emb])
        if (distance[0][0] < merge_threshold or len(base_split_text) + len(split.page_content) > max_chunk_size):
            splits.append(Document(page_content=base_split_text))
            base_split_text = split.page_content
            base_split_emb = split_emb
        else:
            base_split_text += split.page_content

    if base_split_text:
        splits.append(Document(page_content=base_split_text))

    return splits

In [19]:
# hfembeddings = HuggingFaceEmbeddings(model_name="klue/roberta-small")
u_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [20]:
semantic_splits = semantic_chunker(docs, merge_threshold=0.8, embeddings=u_embeddings)

print("SemanticChunker Splits:", len(semantic_splits))

SemanticChunker Splits: 2299


In [21]:
vectorstore = Chroma.from_documents(documents=semantic_splits, embedding=UpstageEmbeddings(model="solar-embedding-1-large"))

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [23]:
tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")

In [24]:
#
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            너는 축구 용어에 대한 질문에 답하는 AI 챗봇이야.
            제공된 문서를 참고하고, 질문 히스토리에 기반해서 답변해줘.
            답변을 모르면 그냥 모른다고 답해줘.
            ---
            CONTEXT:
            {context}
            """
        ), 
        MessagesPlaceholder(variable_name='history'), 
        ("human", "{input}"),
    ]
)

In [25]:
chain = chat_with_history_prompt | llm | StrOutputParser()

In [26]:
def get_relevant_context(question, retriever):
    # retriever를 사용해 관련된 문서만 가져옴
    relevant_docs = retriever.get_relevant_documents(question)
    
    # 관련 문서들을 하나의 문자열로 합침
    return "\n".join(doc.page_content for doc in relevant_docs)


In [27]:
def chat_2(message, history):
    print(message)
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    generator = chain.stream({"message": message, "history": history_langchain_format})

    assistant = ""
    for gen in generator:
        assistant += gen
        yield assistant

In [28]:
def chat(question, history):
    print(question)
    history_langchain_format = []

    #####
    # 기존 질문 및 컨텍스트 처리
    relevant_context = get_relevant_context(question, retriever)
    # 모델 호출
    ai = chain.invoke({
        "history": history_langchain_format, 
        "context": relevant_context, 
        "input": question
    })
    #####

    # for human, ai in history:
    #     history_langchain_format.append(HumanMessage(content=human))
    #     history_langchain_format.append(AIMessage(content=ai))

    #generator = chain.stream({"message": question, "history": history_langchain_format})

    # assistant = ""
    # for gen in generator:
    #     assistant += gen
    #     yield assistant

    return ai

In [47]:
import gradio as gr
from __future__ import annotations
from typing import Iterable
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
class Football(Base):
    def __init__(
        self,
        *,
        primary_hue: colors.Color | str = colors.emerald,
        secondary_hue: colors.Color | str = colors.emerald,
        neutral_hue: colors.Color | str = colors.neutral,
        spacing_size: sizes.Size | str = sizes.spacing_lg,
        radius_size: sizes.Size | str = sizes.radius_lg,
        text_size: sizes.Size | str = sizes.text_lg,
        font: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Nanum Barun Gothic"),
            "sans-serif",
        ),
        font_mono: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Nanum Barun Gothic"),
            "sans-serif",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )
        super().set(
            body_background_fill="#6da682",
            body_background_fill_dark="#6da682",
            
            block_background_fill="#ffffff",
            block_background_fill_dark="#ffffff",
            block_shadow="0 4px 8px rgba(0, 0, 0, 0.1)",
            
            button_primary_background_fill="linear-gradient(90deg, *primary_300, *secondary_400)",
            button_primary_background_fill_hover="linear-gradient(90deg, *primary_200, *secondary_300)",
            button_primary_text_color="black",
            button_primary_background_fill_dark="linear-gradient(90deg, *primary_600, *secondary_800)",
            
            slider_color="*secondary_300",
            slider_color_dark="*secondary_600",
            block_title_text_weight="600",
            block_border_width="2px",
            button_primary_shadow="*shadow_drop_md",
            button_large_padding="24px",
            
            input_background_fill="#ffffff",
            input_background_fill_dark="#ffffff",
            input_shadow="none",
        )

football = Football()

# CSS로 Football 스타일 정의
css = """
/* 전체 배경 설정 */
.gradio-container {
    background-color: #6da682;
}

/* 블록(박스) 설정 및 텍스트에 그림자 추가 */
.gr-block {
    background-color: #ffffff;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    border-radius: 10px; /* 모서리 둥글기 */
    padding: 20px;
    text-shadow: none;
}

/* 버튼 스타일 */
button.primary {
    background: linear-gradient(90deg, #66cdaa, #008b8b);
    color: black;
    padding: 12px 24px;
    border-radius: 8px; /* 버튼 모서리 둥글기 */
    box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.2);
}

button.primary:hover {
    background: linear-gradient(90deg, #50a37c, #006b6b);
}

button.primary:active {
    background: linear-gradient(90deg, #006b6b, #66cdaa);
}

/* 슬라이더 색상 */
.gr-slider .track-fill {
    background-color: #66cdaa;
}

/* 입력 필드 배경 */
input[type="text"], textarea {
    background-color: #ffffff;
    border-radius: 8px; /* 입력 필드 모서리 둥글기 */
    padding: 10px;
    box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.1);
}

/* 메시지 창 전체 배경 설정 */
.gr-chatbot {
    background-color: #ffffff; /* 메시지 창 전체 흰색 배경 */
    border: none;
    padding: 0;
}

/* 전체 배경 이미지 설정 */
.gradio-container {
    background-image: url('https://static.vecteezy.com/system/resources/previews/013/950/541/non_2x/football-field-flat-flat-icon-free-vector.jpg');
    background-size: cover;
    background-position: center; /* 이미지 가운데 정렬 */
    background-repeat: no-repeat; /* 이미지 반복하지 않음 */
}

/* 제목 텍스트에 추가적인 그림자 효과 */
#gradio-animation {
    color: white;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.6); /* 그림자 효과 */
}
"""

# JavaScript 코드
js = """
function createGradioAnimation() {
    var container = document.createElement('div');
    container.id = 'gradio-animation';
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '20px';

    var text = 'FootBot⚽️';
    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.5s';
                letter.innerText = text[i];

                container.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 50);
            }, i * 250);
        })(i);
    }

    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);

    return 'Animation created';
}
"""


In [48]:
from langchain_upstage import ChatUpstage
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from collections import Counter
import requests
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to fetch news articles from Naver Soccer News API
def fetch_articles(api_url, headers):
    response = requests.get(api_url, headers=headers)
    response.raise_for_status()
    articles = response.json()
    return articles['items'][:100]

# Function to clean and tokenize text
def clean_and_tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.lower().split()
    return tokens

# Function to calculate keyword frequency for importance
def calculate_importance(article_text, common_keywords):
    tokens = clean_and_tokenize(article_text)
    keyword_count = sum(1 for token in tokens if token in common_keywords)
    return keyword_count

# Function to remove duplicate articles based on content similarity
def remove_duplicates(articles, threshold=0.8):
    descriptions = [article['description'] for article in articles]
    vectorizer = TfidfVectorizer().fit_transform(descriptions)
    similarity_matrix = cosine_similarity(vectorizer)
    unique_articles = []
    seen_indices = set()

    for i, article in enumerate(articles):
        if i not in seen_indices:
            unique_articles.append(article)
            similar_indices = np.where(similarity_matrix[i] > threshold)[0]
            seen_indices.update(similar_indices)

    return unique_articles

# Fetch, process, and summarize articles
def summarize_articles():
    api_url = "https://openapi.naver.com/v1/search/news.json?query=축구&display=100&sort=date"
    headers = {
        "X-Naver-Client-Id": "aMevyrYH7wOTUGOb7H7l",
        "X-Naver-Client-Secret": "gSdqbjXchL"
    }

    # Fetch and process articles
    articles = fetch_articles(api_url, headers)
    articles = remove_duplicates(articles)

    # Aggregate text to find common keywords
    all_text = " ".join([article['description'] for article in articles if 'description' in article])
    common_tokens = Counter(clean_and_tokenize(all_text)).most_common(20)
    common_keywords = [token for token, _ in common_tokens]

    # Initialize LLM and summarization chain
    llm = ChatUpstage()
    prompt_template = PromptTemplate(input_variables=["article"], template="다음 기사를 요약해 주세요: {article}")
    summary_chain = LLMChain(llm=llm, prompt=prompt_template)

    # Summarize and rank articles by importance
    important_articles = []
    for article in articles:
        if 'description' in article:
            importance_score = calculate_importance(article['description'], common_keywords)
            summary = summary_chain.run(article['description'])
            title_prompt = f"이 기사에 적합한 제목을 지어주세요: {article['description']}"
            title = llm.generate([title_prompt]).generations[0][0].text.strip()
            important_articles.append((title, summary, importance_score))

    # Sort and return top 3 articles
    important_articles.sort(key=lambda x: x[2], reverse=True)
    return important_articles[:3]

# Gradio Interface
def display_summaries():
    articles = summarize_articles()
    summary_md = "\n\n".join(
        [f"### 🤖 **{title}**\n{summary}" for title, summary, _ in articles]
    )
    return summary_md

In [51]:
with gr.Blocks(theme=football, js=js) as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("<h1 style='text-align: center;'>오늘의 축구 뉴스 요약</h1>")
            article_summaries = gr.Markdown(value=display_summaries, elem_id="article_summaries")
        with gr.Column():
            chatbot = gr.ChatInterface(
                chat,
                examples=[
                    "오프사이드는 어떤 상황에서 발생하나요?",
                    "패널티킥은 언제 주어지나요?",
                    "옐로우카드와 레드카드의 차이는 무엇인가요?",
                ],
                title="궁금한 사항을 검색해보세요",
                description="모르는 용어나 규칙을 질문할 수 있어요!",
            )
            chatbot.chatbot.height = 300 

오프사이드는 어떤 상황에서 발생하나요?
옐로우카드와 레드카드의 차이는 무엇인가요?
축구 선수 수는 총 몇명인가요?


Traceback (most recent call last):
  File "/Users/hanjiin/anaconda3/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hanjiin/anaconda3/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hanjiin/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hanjiin/anaconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1565, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hanjiin/anaconda3/lib/python3.11/site-packages/gradio/utils.py", line 813, in async_wrapper
    response = await f(*args, **kwargs)
         

축구에서 무승부는 어떻게 결정하나요?
골킥은 언제 주어지나요?
골킥은 언제 주어지나요?
오프사이드는 어떤 상황에서 발생하나요?


In [50]:
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
